In [ ]:
import os
import random

random.seed(43)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger().setLevel(logging.ERROR)

import torch
import torch.nn as nn
from torchsummary import summary

In [ ]:
# find device
if torch.cuda.is_available(): # NVIDIA
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # apple silicon
    device = torch.device('mps') 
else:
    device = torch.device('cpu') # fallback
device = device = torch.device('cpu')

In [ ]:
num_cpus = os.cpu_count()
print(num_cpus, 'CPUs available')
num_cpus = 1

# Preparing the dataset

In [ ]:
dataai="../data/WISDM_ar_v1.1/"
datapath =  dataai
col_names = ['user', 'activity', 'timestamp', 'x-accel', 'y-accel', 'z-accel']

df = pd.read_csv(datapath+"WISDM_ar_v1.1_raw.txt",
                  header=None, names=col_names, delimiter=',', comment=';',
                    on_bad_lines='skip') #skip/warn bad lines
print(df.shape)
df.head()

In [ ]:
num_usrs = df['user'].unique()
print(num_usrs.shape)

In [ ]:
class_labels = df.activity.unique()
num_channels =3 # x-accel, y-accel, z-accel
num_classes = len(class_labels)
print(class_labels)

In [ ]:
activity_to_label = {activity: i for i, activity in enumerate(df['activity'].unique())}
print(activity_to_label)

In [ ]:
# Group by users and activity
grouped = df.groupby(['user', 'activity'])

In [ ]:
# Combine X, Y, Z into a single time-series for each group
time_series = []
labels = []
activity_to_label = {activity: i for i, activity in enumerate(df['activity'].unique())}

for (user, activity), group in grouped:
    # Stack X, Y, Z into a single array of shape (timesteps, 3)
    series = np.column_stack((group['x-accel'], group['y-accel'], group['z-accel']))
    time_series.append(series)
    labels.append(activity_to_label[activity])

print(time_series[0].shape)
print(len(labels), np.unique(labels, return_counts=True))

In [ ]:
# Pad/truncate to a fixed length
max_length = 4*1000  # Choose a fixed length
padded_series = nn.utils.rnn.pad_sequence([torch.tensor(series, dtype=torch.float32) for series in time_series],
                             batch_first=True, padding_value=0)
print(padded_series.shape)
padded_series = padded_series[:, :max_length, :]  # Truncate to max_length if necessary


# Normalize (not needed here, but keep it for plotting)
# padded_series = nn.functional.normalize(padded_series, dim=2, p=2)
print(padded_series.shape, len(labels))


In [ ]:
# add segments
segment_duration = 100
# Calculate the number of segments per sample
num_segments = padded_series.shape[1] // segment_duration
print(" shape of all segmented data", padded_series.shape[0]*num_segments, segment_duration, padded_series.shape[2])
reshaped_data = padded_series.reshape(padded_series.shape[0], num_segments, segment_duration, padded_series.shape[2])

valid_segments = []
valid_labels = []

# Iterate over each sample and its segments to remove segments with all zeros x,y,z
for sample_idx in range(reshaped_data.shape[0]):
    for segment_idx in range(reshaped_data.shape[1]):
        segment = reshaped_data[sample_idx, segment_idx]
        if not (torch.all(segment[:, 0]==0) and torch.all(segment[:, 1]==0) and torch.all(segment[:, 2]==0) ):
            valid_segments.append(segment)
            valid_labels.append(labels[sample_idx])

padded_series = torch.stack(valid_segments)
labels = torch.tensor(valid_labels)
max_length = segment_duration
print(" shape of segmented data with nonzero features",padded_series.shape, labels.shape)
 



In [ ]:
ynumpy = np.array(labels)
class_weights=class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(ynumpy), y=ynumpy) 
class_weights=torch.tensor(class_weights,dtype=torch.float)

print(np.unique(ynumpy),class_weights)
print(class_weights.sum(axis=0))

In [ ]:
# Normalized series
num= 178
fig, ax = plt.subplots(5, 1, figsize=(6, 8))
i=0
for num in [1, 50, 100, 150, 170]:
    print(padded_series[num][99])
    ax[i].plot(padded_series[num][:, 0],
            label=f'X:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].plot(padded_series[num][:, 1],
            label=f'Y:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].plot(padded_series[num][:, 2],
            label=f'Z:{list(activity_to_label.keys())[list(activity_to_label.values()).index(labels[num])]}')
    ax[i].legend()

    i+=1

In [ ]:
from Transformer import run_training_testing_transformer

# print(padded_series.shape)
run_training_testing_transformer.run_transformer_training(padded_series, labels, class_labels, 
                                                  device, num_channels, num_classes,test_size=0.3,
                                                  val_size=0.1, batch_size=8, num_cpus=1,lr=0.001, 
                                                  num_epochs=10,  patience=5, modeltype = "trans1",
                                                  max_length_series=segment_duration, embed_size = 16, nhead = 8,
                                                  dim_feedforward = 32, num_encoderlayers=4, dropout = 0.0, conv1d_emb = True,
                                                  conv1d_kernel_size =7, size_linear_layers = 16,
                                                  opt="adamW", verbose=False, 
                                                  pathsave="./Figs/", weights=None, norm_type="per-channel")